In [439]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

In [531]:
df= pd.read_csv('Job_titles_to_clean.csv')

### Regular expressions to clean the salary column

In [ ]:
def clean_salary(salary):
    pattern = re.compile(".*\$\s*([\d,]+).*\$\s*([\d,]+)")
    matches = re.match(pattern, salary)
    if matches:
        salaries = [int(x.replace(',','')) for x in matches.groups()]
        return np.average(salaries)
    else:
        pattern = re.compile(".*\$\s*([\d,]+)")
        matches = re.match(pattern, salary)
        if matches:
            return float(matches.group(1).replace(',',''))
        else:
            return None

df['clean_salary'] = df.salary.map(lambda x: clean_salary(x))

In [532]:
df.clean_salary.replace(to_replace = 160.0 , value = 165000, inplace=True)

In [533]:
df['job_title'] = df['job_title'].map(lambda x: x if type(x)!=str else x.lower())

### Categorising job titles using regular expressions

In [534]:
def clean_text(text):
    pat = re.compile('.*analy(tics|st)+.*')
    matches = re.match(pat, text)
    if matches:
        return 'analyst'
    else:
        return None

In [535]:
df['analyst'] = df.job_title.map(lambda x: clean_text(x))

In [536]:
def clean_text2(text):
    pat = re.compile('.*data\sscien(ce|tist)+')
    matches = re.match(pat, text)
    if matches:
        return 'data scientist'
    else:
        return None

In [538]:
df['data_scientist']=df.job_title.map(lambda x: clean_text2(x))

In [540]:
df.data_scientist.value_counts()

data scientist    131
Name: data_scientist, dtype: int64

In [541]:
df.analyst.value_counts()

analyst    196
Name: analyst, dtype: int64

In [542]:
df.job_title.value_counts()

data scientist                                                                 36
senior data scientist                                                          16
business analyst                                                               12
business intelligence developer                                                11
software engineer                                                               9
bi developer                                                                    7
business intelligence analyst                                                   7
data engineer                                                                   7
data analyst                                                                    6
business intelligence lead                                                      6
postdoctoral research fellow                                                    4
senior data analyst                                                             4
big data enginee

### Senior role

In [537]:
#df['senior_role'] = df['job_title'].str.contains('lead') | df['job_title'].str.contains('senior')

In [ ]:
df['senior_role'] = (
    df['job_title'].str.contains('lead') | 
    df['job_title'].str.contains('senior') | 
    df['job_title'].str.contains('head') |
    df['job_title'].str.contains('director')    
)

### Categorising salary rates

In [539]:
per_day_pats = ['(per\s+)?day', 'p\.?d']
per_hour_pats = ['(per\s)?hour','p[\s\.]?h']

In [543]:
def categorise_salary(num):
    matches = any([re.match(re.compile('.*' + pat), str(num)) for pat in per_day_pats])
    if matches:
        return 'daily'
    matches = any([re.match(re.compile('.*' + pat), str(num)) for pat in per_hour_pats])
    if matches:
        return 'hourly'
    return 'annual'

In [544]:
df['pay_type'] = df.salary.map(lambda x: categorise_salary(x))

In [545]:
df.pay_type.value_counts(dropna=False)

annual    779
daily      39
hourly     21
Name: pay_type, dtype: int64

In [549]:
#df.drop(['salary'], axis=1, inplace=True)
df.drop(['index','salary'], axis=1, inplace=True)

### Converting salary into annual rate

In [579]:
def salary_multiplier(pay_type):
    if pay_type == 'daily':
        return 250
    elif pay_type == 'hourly':
        return 8*250
    else:
        return 1

In [580]:
df['multiplier'] = df.pay_type.map(salary_multiplier)

In [581]:
df['salary'] = df.clean_salary * df.multiplier

In [582]:
df.drop(['clean_salary','multiplier'], axis=1, inplace=True)

In [583]:
df.head()

,job_title,location,company,class,job_type,description,analyst,senior_role,data_scientist,pay_type,salary
0,data scientist | smart cities,Sydney,Talenza,Science & Healthcare,Contract/Temp,\r\r\nTalenza has been engaged by an Icon Aust...,None,False,data scientist,daily,262500.0
1,data scientist l machine learning engineer l $...,Sydney,NaN,Information & Communication Technology,Full Time,Great team environment/ office space Perman...,None,False,data scientist,annual,154999.5
2,data scientist,Melbourne,FourQuarters Recruitment,Information & Communication Technology,Full Time,Data Scientist | Melbourne CBD | Permanent\r\r...,None,False,data scientist,annual,NaN
3,lead data scientist | python | machine learning,Sydney,Talenza,Science & Healthcare,Contract/Temp,\r\r\nTalenza has been engaged by industry lea...,None,True,data scientist,daily,356250.0
4,data scientist,Sydney,Greythorn Experis,Information & Communication Technology,Contract/Temp,\r\r\n\r\r\n\r\r\nHands-on experience with com...,None,False,data scientist,annual,NaN


### Median salary values 

In [584]:
df.groupby(['analyst'])['salary'].median()

analyst
analyst    125000.0
Name: salary, dtype: float64

In [585]:
df.groupby(['data_scientist'])['salary'].median()

data_scientist
data scientist    150000.0
Name: salary, dtype: float64

In [586]:
df.groupby(['senior_role'])['salary'].median()

senior_role
False    130000.0
True     140000.0
Name: salary, dtype: float64

In [498]:
df.salary.value_counts(dropna=False)

NaN          590
 140000.0     14
 150000.0     13
 200000.0     11
 135000.0      9
 125000.0      8
 115000.0      8
 110000.0      8
 130000.0      8
 120000.0      7
 100000.0      6
 175000.0      5
 160000.0      4
 212500.0      4
 225000.0      4
 180000.0      4
 122500.0      3
 105000.0      3
 154999.5      3
 350000.0      2
 106316.0      2
 168750.0      2
 86000.0       2
 128185.0      2
 77499.5       2
 230000.0      2
 79262.0       2
 80000.0       2
 90000.0       2
 215000.0      2
            ... 
 1450.0        1
 92500.0       1
 248750.0      1
 356250.0      1
 87000.0       1
 118750.0      1
 200125.0      1
 105055.0      1
 100065.0      1
 82499.5       1
 188550.0      1
 123074.5      1
 95000.0       1
 109999.5      1
 104687.0      1
 325000.0      1
 91520.5       1
 187500.0      1
 75000.0       1
 75062.5       1
 145000.0      1
 67500.0       1
 155000.0      1
 83646.0       1
 59999.5       1
 80191.5       1
 113369.0      1
 700.0        

In [500]:
df.shape

(822, 11)

In [588]:
df.tail()

,job_title,location,company,class,job_type,description,analyst,senior_role,data_scientist,pay_type,salary
817,solutions architect (active directory & azure),Perth,FinXL IT Professional Services,Information & Communication Technology,Full Time,\r\r\nThe CompanyThis is a fantastic opportuni...,None,False,None,annual,NaN
818,principal data specialist,Sydney,Data Addiction Pty Ltd,Information & Communication Technology,Full Time,Principal Data Specialist In the realms of dat...,None,False,None,annual,140000.0
819,financial controller,Melbourne,Radisson on Flagstaff Gardens,Financial Services,Full Time,\r\r\nRadisson on Flagstaff Gardens Melbourne ...,None,False,None,annual,NaN
820,data analytics / bi manager,Melbourne,Aginic,Information & Communication Technology,Full Time,THE COMPANY Aginic is a rapidly growing data s...,analyst,False,None,annual,NaN
821,business support officer / junior business ana...,Brisbane,Hudson,Information & Communication Technology,Contract/Temp,\r\r\nOur client is a high profile Queensland ...,analyst,False,None,hourly,87000.0


In [590]:
df.to_csv('almost_cleaned_file.csv',index=False)